brandon.mac.butler@gmail.com

##Blockchain explorers

- Etherchain (https://www.etherchain.org/)
- Etherscan (https://etherscan.io/txs)
- Etherapps
- Ethercamp

http://cryptomining-blog.com/tag/etherscan/

I prefer to use etherchain because it has a nice UI and the API is intuitive to use with adequate documentation

##Etherchain blockchain explorer API

- Etherchain website: https://www.etherchain.org/
- Etherchain API info: https://www.etherchain.org/documentation/api/

Transactions are returned according to their hash

In [163]:
tx_hash = '0x4665af7950cf769eb1c07a202e9dceccbb83e0eca5265d4452a7ba20f66b9b2d'

In [164]:
offset = 5
count = 5

In [176]:
BASE_URL = 'https://etherchain.org/api/txs/{}/{}'.format(5, 5)
#offset: The number of txs to skip
#count: The number of txs to return (max 100 blocks)

In [177]:
import requests

In [178]:
r = requests.get(BASE_URL)

In [179]:
r.status_code

200

In [181]:
#Convert conent into a dictionary
tx = r.json()['data'][0]

In [182]:
#List returns fields 
tx.keys()

[u'hash',
 u'sender',
 u'blockHash',
 u'type',
 u'price',
 u'gasLimit',
 u'gasUsed',
 u'isContractTx',
 u'newContract',
 u'amount',
 u'time',
 u'txIndex',
 u'recipient',
 u'parentHash',
 u'accountNonce',
 u'block_id']

In [183]:
tx

{u'accountNonce': u'1675289',
 u'amount': 0,
 u'blockHash': u'0xcca48a304c320fdf292654c63b6225988c20a21cae8aa9d54552486bf44389e0',
 u'block_id': 4292324,
 u'gasLimit': 115676,
 u'gasUsed': 680,
 u'hash': u'0xf817792c10dfed9c3944b028a62e0493a1ddcf2c2dda2ab047a13b2815856de8',
 u'isContractTx': None,
 u'newContract': 0,
 u'parentHash': u'0x70f0a6c36fb7d4f93086e6ca2440f209b548d7b6b86f44ff0f50d219635ef0ba',
 u'price': 51000000000,
 u'recipient': u'0xa3c1e324ca1ce40db73ed6026c4a177f099b5770',
 u'sender': u'0x43fcb8bebf287d7b2e808cb14650aee501ef5435',
 u'time': u'2017-09-19T20:40:34.000Z',
 u'txIndex': 3,
 u'type': u'call'}

##Set up mongo database to store transaction data

In [10]:
# import MongoDB modules
from pymongo import MongoClient
import requests
import time

# connect to the hosted MongoDB instance
client = MongoClient()
db = client['etherchain']
collection = db['new_transactions']
collection.create_index([('hash', 1)], unique=True)

u'hash_1'

In [11]:
def collect_data(count, offset):
    """
    DESCRIPTION:
        Collects blockchain transaction data from etherchain.org API
        https://etherchain.org/documentation/api/
    
    INPUT:
        - offset: the number of txs to skip
        - count: the number of txs to return (max 100 blocks per request)
        - sleeptime: number of seconds to sleep between api requests
    OUTPUT:
        - stores each blockchain transaction as a document in a MongoDB collection
    """
    
    BASE_URL = 'https://etherchain.org/api/txs/{}/{}'.format(offset, count)
    r = requests.get(BASE_URL)

    if r.status_code != 200:
        print('status code = {}'.format(r.status_code))

    else:
    # store each transaction block in our mongo database
        r = convert_price(r.json())

        for row in range(count):
            try:

                collection.insert_one(r['data'][row])

            except Exception as e:
                #print e
                #print(r['data'][row])
                pass
        
def convert_price(json_dict):
    
    for tx in json_dict['data']:
        tx['amount'] = float(tx['amount'])
        tx['price'] = float(tx['price'])
    
    return json_dict

def call_api():
    
    offset = 2214000
    
    while True:
        collect_data(100, offset)
        offset += 200
        #print(offset)
        if offset % 1000 == 0:
            print('offset: {}'.format(offset))
            
        time.sleep(10)

In [12]:
collection.count()

849499

In [13]:
call_api()

offset: 2215000
offset: 2216000
offset: 2217000
offset: 2218000
offset: 2219000
offset: 2220000
offset: 2221000
offset: 2222000
offset: 2223000
offset: 2224000
offset: 2225000
offset: 2226000
offset: 2227000
offset: 2228000
offset: 2229000
offset: 2230000
offset: 2231000
offset: 2232000
offset: 2233000
offset: 2234000
offset: 2235000
offset: 2236000
offset: 2237000
offset: 2238000
offset: 2239000
offset: 2240000
offset: 2241000
offset: 2242000
offset: 2243000
offset: 2244000
offset: 2245000
offset: 2246000
offset: 2247000
offset: 2248000
offset: 2249000
offset: 2250000
offset: 2251000
offset: 2252000
offset: 2253000
offset: 2254000
offset: 2255000
offset: 2256000
offset: 2257000
offset: 2258000
offset: 2259000
offset: 2260000
offset: 2261000
offset: 2262000
offset: 2263000
offset: 2264000
offset: 2265000
offset: 2266000
offset: 2267000
offset: 2268000
offset: 2269000
offset: 2270000
offset: 2271000
offset: 2272000
offset: 2273000
offset: 2274000
offset: 2275000
offset: 2276000
offset: 

KeyboardInterrupt: 

#Get tx for block to get tx for missing blocks 

In [12]:
r = requests.get("https://etherchain.org/api/block/4292736/tx")

In [13]:
r.status_code

200

In [16]:
r.content

'{"status":1,"data":[{"hash":"0x0271926268de3ae621d8d5f1736e30eacbeee696fa2d1f4e6e91e9608246a193","sender":"0x1b7712d01b363cd75e4d3696478b151846ad332a","recipient":"0x8d12a197cb00d4747a1fe03395095ce2a5cc6819","accountNonce":"9","price":10000000000,"gasLimit":250000,"amount":7000000000000000000,"block_id":4292736,"time":"2017-09-19T23:19:05.000Z","newContract":0,"isContractTx":null,"blockHash":"0xf74762ca38a80bded6d5911fa8fe922e41fa4fce51dc52ee54a53201a0840d7b","parentHash":"0x0271926268de3ae621d8d5f1736e30eacbeee696fa2d1f4e6e91e9608246a193","txIndex":null,"gasUsed":44493,"type":"tx"},{"hash":"0x072ad25275ab00556c225e8f84d2feb2af7779862eb06e287ebcdc5a4228e9fc","sender":"0x8dd9d4593a3ee1af08ef1a8fd24427991605cd5a","recipient":"0x8a99ed8a1b204903ee46e733f2c1286f6d20b177","accountNonce":"0","price":4000000000,"gasLimit":90000,"amount":0,"block_id":4292736,"time":"2017-09-19T23:19:05.000Z","newContract":0,"isContractTx":null,"blockHash":"0xf74762ca38a80bded6d5911fa8fe922e41fa4fce51dc52ee54a

In [17]:
r = convert_price(r.json())

In [48]:
len(r['data'])

185

In [51]:
def collect_blocks(block_id):
    """
    DESCRIPTION:
        Collects blockchain transaction data from etherchain.org API
        https://etherchain.org/documentation/api/
    
    INPUT:
        - offset: the number of txs to skip
        - count: the number of txs to return (max 100 blocks per request)
        - sleeptime: number of seconds to sleep between api requests
    OUTPUT:
        - stores each blockchain transaction as a document in a MongoDB collection
    """
    
    BASE_URL = 'https://etherchain.org/api/block/{}/tx'.format(block_id)
    r = requests.get(BASE_URL)

    if r.status_code != 200:
        print('status code = {}'.format(r.status_code))

    else:
    # store each transaction block in our mongo database
        r = convert_price(r.json())

        for row in range(len(r['data'])):
            try:

                collection.insert_one(r['data'][row])

            except Exception as e:
                print e
                #print(r['data'][row])
                #pass
        
def convert_price(json_dict):
    
    for tx in json_dict['data']:
        tx['amount'] = float(tx['amount'])
        tx['price'] = float(tx['price'])
    
    return json_dict

def call_api_blocks(block_id):
        
    while True:
        collect_blocks(block_id)
            
        time.sleep(5)

In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('../ethereum_blockchain/missing') 

In [35]:
df.missing.values.shape

(340,)

In [43]:
collection.count()

1000527

In [52]:
for i in df.missing.values:
    call_api_blocks(i)

E11000 duplicate key error collection: etherchain.transactions index: hash_1 dup key: { : "0x3acf0fd610d168bb4d56b3cfe7c22817b827b778474d3931657b583d8e20510c" }
E11000 duplicate key error collection: etherchain.transactions index: hash_1 dup key: { : "0x701c48fcac34d6c7d50a6f5b04b2baf3025b68e57d683a313a52c11d9a06637e" }
E11000 duplicate key error collection: etherchain.transactions index: hash_1 dup key: { : "0xa494360921ec30f11bcf9c2aba63b78e76f32474fa344388657f06a1acc424d6" }
E11000 duplicate key error collection: etherchain.transactions index: hash_1 dup key: { : "0x3acf0fd610d168bb4d56b3cfe7c22817b827b778474d3931657b583d8e20510c" }
E11000 duplicate key error collection: etherchain.transactions index: hash_1 dup key: { : "0x701c48fcac34d6c7d50a6f5b04b2baf3025b68e57d683a313a52c11d9a06637e" }
E11000 duplicate key error collection: etherchain.transactions index: hash_1 dup key: { : "0xa494360921ec30f11bcf9c2aba63b78e76f32474fa344388657f06a1acc424d6" }
E11000 duplicate key error collect

KeyboardInterrupt: 